In [157]:
import numpy as np
import matplotlib.pyplot as plt

##### basic matrix multiplication - just to make sure shapes are right

In [158]:
previous_position = np.array([0.5,0.75,1.25,1.5]).reshape(-1,1)
binomial_moves = np.array([50,-50]).reshape(-1,1)
previous_pnl = np.array([87.5,12.5,12.5,-112.5]).reshape(-1,1)

In [159]:
print(np.shape(previous_position),np.shape(binomial_moves),np.shape(previous_pnl))

(4, 1) (2, 1) (4, 1)


In [160]:
(np.dot(previous_position,np.transpose(binomial_moves)) + previous_pnl)

array([[ 112.5,   62.5],
       [  50. ,  -25. ],
       [  75. ,  -50. ],
       [ -37.5, -187.5]])

In [161]:
next_pnl = (np.dot(previous_position,np.transpose(binomial_moves)) + previous_pnl).reshape(-1,1)

In [162]:
next_pnl

array([[ 112.5],
       [  62.5],
       [  50. ],
       [ -25. ],
       [  75. ],
       [ -50. ],
       [ -37.5],
       [-187.5]])

## builder functions

In [163]:
def current_pnl(previous_position,binomial_moves,previous_pnl):
    previous_position = previous_position.reshape(-1,1)
    binomial_moves = binomial_moves.reshape(-1,1)
    previous_pnl = previous_pnl.reshape(-1,1)
    
    return (np.dot(previous_position,np.transpose(binomial_moves)) + previous_pnl).reshape(-1,1)

In [164]:
def current_position(previous_position,buy_size,sell_size):
    lot_size = np.array([-sell_size,buy_size]).reshape(-1,1)
    previous_position = previous_position.reshape(-1,1)
    return (previous_position + np.transpose(lot_size)).reshape(-1,1)


In [165]:
previous_position = np.array([0.75,1.25]).reshape(-1,1)
buy_size = 0.25
sell_size = 0.25

In [166]:
current_position(previous_position,buy_size,sell_size)

array([[0.5],
       [1. ],
       [1. ],
       [1.5]])

In [167]:
def time_step(previous_position,binomial_moves,previous_pnl,buy_size,sell_size):
    return current_position(previous_position,buy_size,sell_size),current_pnl(previous_position,binomial_moves,previous_pnl)

## without increasing position sizing

In [168]:
t = 3 # number of time steps
starting_bet = 1000000
previous_pnl = np.array([0])
previous_position = np.array([starting_bet])
buy_size = 250000
sell_size = 250000
binomial_moves = np.array([50,-50]).reshape(-1,1)/10000

In [169]:
def expand_array(arr, num_copies):
    # Repeat the array side by side for user-defined times
    duplicated_arr = np.tile(arr, (1, num_copies))
    # Flatten the array
    vertical_arr = duplicated_arr.reshape(-1,1)
    return vertical_arr


In [170]:
def binomial_tree(previous_position,binomial_moves,previous_pnl,buy_size,sell_size,t):
    pnl_df = pd.DataFrame(np.zeros((2**t,t)),columns = list(['pnl'+str(i+1) for i in range(0,t)]))
    position_df = pd.DataFrame(np.zeros((2**t,t)),columns = list(['position'+str(i+1) for i in range(0,t)]))
    for i in range (0,t):
        cur_pos = previous_position
        previous_position,previous_pnl = time_step(cur_pos,binomial_moves,previous_pnl,buy_size,sell_size)
        pnl_df.iloc[:,i] = expand_array(previous_pnl, int(2**t/len(previous_pnl)))
        position_df.iloc[:,i] = expand_array(cur_pos, int(2**t/len(cur_pos)))
    return  pnl_df,position_df


In [171]:
pnl,position=binomial_tree(previous_position,binomial_moves,previous_pnl,buy_size,sell_size,t)

In [172]:
print(position)
print(pnl)

   position1  position2  position3
0    1000000   750000.0   500000.0
1    1000000   750000.0   500000.0
2    1000000   750000.0  1000000.0
3    1000000   750000.0  1000000.0
4    1000000  1250000.0  1000000.0
5    1000000  1250000.0  1000000.0
6    1000000  1250000.0  1500000.0
7    1000000  1250000.0  1500000.0
     pnl1     pnl2     pnl3
0  5000.0   8750.0  11250.0
1  5000.0   8750.0   6250.0
2  5000.0   1250.0   6250.0
3  5000.0   1250.0  -3750.0
4 -5000.0   1250.0   6250.0
5 -5000.0   1250.0  -3750.0
6 -5000.0 -11250.0  -3750.0
7 -5000.0 -11250.0 -18750.0


In [173]:
def calc_expected(previous_pnl):
    print('expected value:-',np.sum(previous_pnl)/len(previous_pnl))

def calc_pop(previous_pnl):
    profit = np.where(previous_pnl>0,1,0)
    pop = np.sum(profit)/len(profit)
    print('probability of profit:- ',pop*100,'%')

In [174]:
last_pnl = pnl.iloc[:,-1]
calc_expected(last_pnl)
calc_pop(last_pnl)

expected value:- 0.0
probability of profit:-  50.0 %


we see if we just buy and sell the same amount we get expected value of 0. we also have a probability of profit of 50 %

## While increasing position sizing

In [175]:
t = 3
starting_bet = 1000000
previous_pnl = np.array([0])
previous_position = np.array([starting_bet])
buy_size = 250000
sell_size = 250000
binomial_moves = np.array([50,-50]).reshape(-1,1)/10000

bet_increase_ratio = 5 # we set it as 5 to correspond with your example - postion size at time 2 or t=1 is 1.25M whereas at t=0 it is 0.25M. 
# keep in mind it will continue to increase for later time steps to solve this change i>= 1 to i==1 in the if statement below

In [176]:
def biased_binomial_tree(previous_position,binomial_moves,previous_pnl,buy_size,sell_size,t,bet_increase_ratio):
    pnl_df = pd.DataFrame(np.zeros((2**t,t)),columns = list(['pnl'+str(i+1) for i in range(0,t)]))
    position_df = pd.DataFrame(np.zeros((2**t,t)),columns = list(['position'+str(i+1) for i in range(0,t)]))
    for i in range (0,t):
        if i>=1:  # increase bet size after first time step
            buy_size = buy_size * bet_increase_ratio # we increase every subsequent time step but not relistic as we dont have infinite money
            #sell_size = sell_size*bet_increase_ratio - uncomment if we have a bearish outlook ahead and comment out above line.
        cur_pos = previous_position
        previous_position,previous_pnl = time_step(cur_pos,binomial_moves,previous_pnl,buy_size,sell_size)
        pnl_df.iloc[:,i] = expand_array(previous_pnl, int(2**t/len(previous_pnl)))
        position_df.iloc[:,i] = expand_array(cur_pos, int(2**t/len(cur_pos)))
    return pnl_df,position_df

In [177]:
pnl,position=biased_binomial_tree(previous_position,binomial_moves,previous_pnl,buy_size,sell_size,t,bet_increase_ratio)

In [178]:
print(position)
print(pnl)

   position1  position2  position3
0    1000000   750000.0   500000.0
1    1000000   750000.0   500000.0
2    1000000   750000.0  2000000.0
3    1000000   750000.0  2000000.0
4    1000000  1250000.0  1000000.0
5    1000000  1250000.0  1000000.0
6    1000000  1250000.0  2500000.0
7    1000000  1250000.0  2500000.0
     pnl1     pnl2     pnl3
0  5000.0   8750.0  11250.0
1  5000.0   8750.0   6250.0
2  5000.0   1250.0  11250.0
3  5000.0   1250.0  -8750.0
4 -5000.0   1250.0   6250.0
5 -5000.0   1250.0  -3750.0
6 -5000.0 -11250.0   1250.0
7 -5000.0 -11250.0 -23750.0


In [179]:
last_pnl = pnl.iloc[:,-1]
calc_expected(last_pnl)
calc_pop(last_pnl)

expected value:- 0.0
probability of profit:-  62.5 %
